In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy 
import re
import string
from tqdm.auto import tqdm  

In [85]:
fake_news_df = pd.read_csv("/home/raghu/Downloads/Datasets/fake_news_detection/Fake.csv")
true_news_df = pd.read_csv("/home/raghu/Downloads/Datasets/fake_news_detection/True.csv")

In [ ]:
#pandas is used to analysing the date fast but pycaret is used to speed up making of model fast


In [ ]:
fake_news_df.head() 

In [ ]:
fake_news_df['subject'].value_counts() 

In [87]:
fake_news_df.drop("date",axis=1,inplace=True)

In [86]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
fake_news_df['subject'] = label_encoder.fit_transform(fake_news_df['subject'])

In [ ]:
fake_news_df.head()

In [ ]:
fake_news_df.isna().sum()
#our datasets doesnot contain any null values
#then we proceed futher

In [ ]:
fake_news_df.duplicated().sum()

In [ ]:
#dropping the duplicate values
fake_news_df.drop_duplicates(inplace=True)

In [ ]:
true_news_df.duplicated().sum()
true_news_df.drop_duplicates(inplace=True)

In [88]:
fake_news_df['is_fake'] = 1
true_news_df['is_fake'] = 0
news = pd.concat([fake_news_df,true_news_df],ignore_index=True)

In [89]:
news.drop(['date','subject'],axis=1, inplace=True)
news 

,title,text,is_fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,1
...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,0


In [ ]:
nlp = spacy.load('en_core_web_sm')
#stop words:- words which are commonly used in the paragraph
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
def preprocess_text(text):
    # Parse the text with Spacy
    doc = nlp(text)
    
    # Lemmatize the tokens and remove stop words
    lemmas = [token.lemma_.lower() for token in doc if not token.is_stop]
    
    # Join the lemmas back into a string and return it
    return " ".join(lemmas)

In [ ]:
tqdm.pandas()
news['text'] = news['text'].progress_apply(preprocess_text) 

In [71]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(news['text'],news['is_fake'],test_size = 0.2,random_state=42)

In [74]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x_train_vect = vectorizer.fit_transform(x_train)
x_test_vect = vectorizer.transform(x_test)
#vectorization of the sentences

In [75]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [76]:
clf= MultinomialNB()
clf

MultinomialNB()

In [78]:
clf.fit(x_train_vect,y_train)

MultinomialNB()

In [79]:
y_pred = clf.predict(x_test_vect) 

In [80]:
y_pred   

array([1, 0, 0, ..., 1, 0, 1])

In [81]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred) 
print(cm)

[[4016  184]
 [ 222 4516]]


In [82]:
acc = accuracy_score(y_test,y_pred) 
print("accuracy:",acc)

accuracy: 0.9545759677780264
